In [1]:
#to do
#under construction
import os
import math
import torch
import torch.utils.data
import numpy as np
import json

class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
            np.int16, np.int32, np.int64, np.uint8,
            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32, 
            np.float64)):
            return float(obj)
        elif isinstance(obj,(np.ndarray,)): #### This is the fix
            return obj.tolist()
        return json.JSONEncoder.default(self, obj) 
    
def save_to_json(dic,target_dir):
    dumped = json.dumps(dic, cls=NumpyEncoder)  
    file = open(target_dir, 'w')  
    json.dump(dumped, file)
    file.close()
    
def read_from_json(target_dir):
    f = open(target_dir,'r')
    data = json.load(f)
    data = json.loads(data)
    f.close()
    return data 

In [3]:
file_location = "ballet_revised_pose_pairs.json"
pose_dict=read_from_json(file_location)

In [5]:
# for each key, it is a video
print(sorted(pose_dict.keys()))

['000', '001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016', '017', '018', '019', '020', '021', '022', '023', '024', '025', '026', '027', '028', '029', '030', '031', '032', '033', '034', '035', '036', '037', '038', '039']


In [6]:
# for each key, it imply the beginning time. Eg, "025" is the 25s of the video. 
#Each key includes 10s length of dance and music
print(sorted(pose_dict["000"].keys()))

['025', '035', '045', '055', '065', '075', '085', '095', '105', '115', '125', '135', '145']


In [7]:
print(sorted(pose_dict["000"]["025"].keys()))

['audio_sequence', 'joint_coors']


In [8]:
length=0
keys=sorted(pose_dict.keys())
for key in keys:
    #index = str("%03d" % i)
    sub_keys=sorted(pose_dict[str(key)].keys())
    if key=="046":
        break
    for sub_key in sub_keys:
        temp_pose=np.array(pose_dict[str(key)][str(sub_key)]["joint_coors"])
        if(temp_pose.shape==(100,)):
            print("girl"+key+" "+sub_key+" is wrong")
            continue
        length+=1

target=torch.FloatTensor(2*length,50,1600).zero_()
label=torch.FloatTensor(2*length,50,18,2).zero_()
index=0

keys=sorted(pose_dict.keys())
#keys=["017","018"]
for key in keys:
    #index = str("%03d" % i)
    sub_keys=sorted(pose_dict[str(key)].keys())
    if key=="046":
        break
    for sub_key in sub_keys:

        print(key+" "+sub_key)
        temp_audio=np.array(pose_dict[str(key)][str(sub_key)]['audio_sequence'])

        temp_pose=np.array(pose_dict[str(key)][str(sub_key)]["joint_coors"])
        if(temp_pose.shape==(100,)):
            continue
        x_coor=(temp_pose[:,:,0]/320)-1
        y_coor=(temp_pose[:,:,1]/180)-1
        temp=np.zeros((100,18,2))
        temp[:,:,0]=x_coor
        temp[:,:,1]=y_coor
        temp_pose=temp

        d = torch.from_numpy(temp_audio).type(torch.LongTensor)

        slices1=d[0:80000].view(50,1600)
        slices2=d[80000:160000].view(50,1600)
        target[index]=slices1
        target[index+1]=slices2

        label[index]=torch.from_numpy(temp_pose[0:50,:,:])
        label[index+1]=torch.from_numpy(temp_pose[50:100,:,:])
        index+=2

girl007 075 is wrong
girl014 025 is wrong
000 025
000 035
000 045
000 055
000 065
000 075
000 085
000 095
000 105
000 115
000 125
000 135
000 145
001 025
001 035
001 045
001 055
001 065
001 075
001 085
001 095
001 105
001 115
001 125
001 135
001 145
001 155
002 025
002 035
002 045
002 055
002 065
002 075
002 085
002 095
002 105
002 115
002 125
002 135
002 145
003 025
003 035
003 045
003 055
003 065
003 075
003 085
003 095
003 105
003 115
003 125
004 025
004 035
004 045
004 055
004 065
004 075
005 025
005 035
005 045
005 055
005 065
006 025
006 035
006 045
006 055
006 065
007 025
007 035
007 045
007 055
007 065
007 075
007 085
007 095
007 105
007 115
008 025
008 035
008 045
008 055
008 065
008 075
008 085
008 095
008 105
009 025
009 035
009 045
009 055
009 065
009 075
009 085
009 095
010 025
010 035
010 045
010 055
011 025
011 035
011 045
011 055
011 065
011 075
011 085
011 095
011 105
011 115
012 025
012 035
012 045
012 055
012 065
012 075
012 085
012 095
013 025
013 035
013 045
013 05

In [9]:
print("the shape of the audio", target.shape)
print("the shape of the dance sequence", label.shape)

the shape of the audio torch.Size([686, 50, 1600])
the shape of the dance sequence torch.Size([686, 50, 18, 2])
